<a href="https://colab.research.google.com/github/evysb/RL_Codes/blob/main/Q_Learning/Q_Learning_FrozenLake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q* Learning with FrozenLake 4x4 

Neste Notebook, implementaremos um agente <b> que reproduz FrozenLake. </b>

![alt text](http://simoninithomas.com/drlc/Qlearning/frozenlake4x4.png)

O objetivo deste jogo é <b> ir do estado inicial (S) para o estado objetivo (G) </b> caminhando apenas em ladrilhos congelados (F) e evitando buracos (H). No entanto, o gelo é escorregadio, ** então você nem sempre se moverá na direção que pretende (ambiente estocástico) **

By [lukewys](https://github.com/lukewys)
And [simoninithomas](https://github.com/simoninithomas)

## Step -1: Instale as dependências no Google Colab

In [ ]:
!pip install numpy
!pip install gym

## Step 0: Importar as dependências 📚
Usamos 3 bibliotecas:
- `Numpy` para nosso Qtable
- `OpenAI Gym` para nosso ambiente FrozenLake
- `Random` para gerar números aleatórios

In [ ]:
import numpy as np
import gym
import random

## Step 1: Crie o ambiente 🎮
- Aqui vamos criar o ambiente FrozenLake 4x4.
- OpenAI Gym é uma biblioteca <b> composta por vários ambientes que podemos usar para treinar nossos agentes. </b>
- No nosso caso optamos por usar Frozen Lake.

In [ ]:
env = gym.make("FrozenLake-v0")

## Step 2: Crie a tabela Q e inicialize-a 🗄️
- Agora, vamos criar nossa tabela Q, para saber de quantas linhas (estados) e colunas (ações) precisamos, precisamos calcular o action_size e o state_size
- OpenAI Gym nos fornece uma maneira de fazer isso: `env.action_space.n` e` env.observation_space.n`

In [ ]:
action_size = env.action_space.n
state_size = env.observation_space.n

In [ ]:
state_size

16

In [ ]:
# Crie nossa tabela Q com linhas state_size e colunas action_size (64x4)
print('state_size ',state_size)
print('action_size ',action_size)
qtable = np.zeros((state_size, action_size))
print(qtable)

state_size  16
action_size  4
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Step 3: Crie os hiperparâmetros ⚙️
- Aqui, vamos especificar os hiperparâmetros

In [ ]:
total_episodes = 30000       # Total episodes
learning_rate = 0.35          # Learning rate
max_steps = 99               # Max steps per episode
gamma = 0.99                # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.005            # Exponential decay rate for exploration prob

## Step 4: O algoritmo de aprendizagem Q 🧠
- Agora implementamos o algoritmo de aprendizagem Q:
  ![alt text](http://simoninithomas.com/drlc/Qlearning//qtable_algo.png)


In [ ]:
# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])
            #print(exp_exp_tradeoff, "action", action)

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
            #print("action random", action)
            
        
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)
    

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)

Score over time: 0.6267
[[0.53818002 0.48324973 0.49723459 0.48406479]
 [0.26720103 0.31077912 0.16111621 0.50215077]
 [0.36926342 0.35209472 0.30524911 0.44248906]
 [0.32677114 0.31944778 0.13917174 0.40667972]
 [0.56781325 0.23936939 0.45841468 0.43076468]
 [0.         0.         0.         0.        ]
 [0.04798811 0.00961932 0.42548689 0.10532619]
 [0.         0.         0.         0.        ]
 [0.3534624  0.39878702 0.46496338 0.63848207]
 [0.26955977 0.61283535 0.3761658  0.29139295]
 [0.67827298 0.24702084 0.2336198  0.2432556 ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.3535872  0.37143445 0.81420059 0.38146517]
 [0.60821444 0.92136603 0.63934023 0.59840911]
 [0.         0.         0.         0.        ]]


## Step 5: Use nossa tabela Q para jogar FrozenLake! 👾
- Depois de 10.000 episódios, nossa tabela-Q pode ser usada como uma "folha de cheats" para jogar FrozenLake "
- Ao rodar este celular você pode ver nosso agente jogando FrozenLake.

In [ ]:
env.reset()

for episode in range(5):
    state = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            if new_state == 15:
                print("We reached our Goal 🏆")
            else:
                print("We fell into a hole ☠️")
            
            # We print the number of step it took.
            print("Number of steps", step)
            
            break
        state = new_state
env.close()

****************************************************
EPISODE  0
  (Down)
SFFF
FHFH
FFFH
HFFG
We reached our Goal 🏆
Number of steps 10
****************************************************
EPISODE  1
  (Down)
SFFF
FHFH
FFFH
HFFG
We reached our Goal 🏆
Number of steps 68
****************************************************
EPISODE  2
****************************************************
EPISODE  3
  (Down)
SFFF
FHFH
FFFH
HFFG
We reached our Goal 🏆
Number of steps 7
****************************************************
EPISODE  4
  (Down)
SFFF
FHFH
FFFH
HFFG
We reached our Goal 🏆
Number of steps 27
